# Let's try to scrap discogs to check the items for sale of a release

## So far, the tool does that:
1. checks a list of Discogs URLs to watch
2. Scraps data for each of the items for sale of each URL: sale link, price, shipping, ships from
3. Saves the data frame to csv
4. Checks a similar csv of the day before for differences
5. Lists all the data that was not there yestderday and is here today
6. Sends me an email with this data

## Things to add/change:
1. Check the 2nd most recent .csv of the export folder instead of the csv with the name of yesterday
2. Automatte the script to run every hour (to start)
3. (maybe not needed) randomize the time to start checking a page, because Discogs might limit the number of requests we do even though it's not done through the API
4. put credentials in a separate json file
5. fix bug that retrieves 'shipping' instead of the shipping value for some URls
6. add name of the release to the data frame
7. connect the script to a gsheet so that i can add new urls without changing the script

In [1]:
from bs4 import BeautifulSoup
import requests

urls = ['https://www.discogs.com/sell/release/85195?ev=rb'
        , 'https://www.discogs.com/sell/release/8580086?ev=rb'
        # add other URLs here
       ]
# e.g. https://www.discogs.com/sell/release/85195?ev=rb

list_links = [] 
list_prices = []
list_shipping_prices = []
list_countries = []
list_conditions = []

# loop for every url as an input
for url in urls:

    # First let's parse the URL with BeautifulSoup
    source = requests.get(url).text
    soup = BeautifulSoup(source,'lxml')
    body = soup.find('body')
    link = body.find('td',class_="item_add_to_cart hide_mobile").p.strong.a
    
    #PART 1
    # Let's scrap URLs from items for sale

    # parsing
    all_links = body.find_all('td',class_="item_add_to_cart hide_mobile")

    # to append later to the result of the parsing
    generic_url = 'http://www.discogs.com'

    # loop for all the parsed HTML elements
    for n in all_links: 
        link = n.p.strong.a.get('href')
        link = generic_url + link[:link.find('?')]
        list_links.append(link)
        
    # Same for sale price

    all_prices = body.find_all(class_ = 'item_price hide_mobile')

    # loop for all the parsed HTML elements
    for n in all_prices: 
        price = n.span.text
        list_prices.append(price)

         # same for shipping

    all_shipping_prices = body.find_all('td', class_ = 'item_price hide_mobile')

    # loop for all the parsed HTML elements
    for n in all_shipping_prices: 
        shipping_price = n.text
        shipping_price = shipping_price[shipping_price.rfind('+')+1:]

        index = shipping_price.find('\n')
        shipping_price = shipping_price[:index]

        list_shipping_prices.append(shipping_price)
        
        # same for shipping country

    all_countries = body.find_all(class_ = 'seller_info')

    # loop for all the parsed HTML elements
    for n in all_countries: 
        country = n.span.text
        country = n.find_next('li').find_next('li').find_next('li').text
        country = country[country.rfind(':')+1:]
        list_countries.append(country)
        
    all_conditions = body.find_all(class_ = 'item_condition')

    # doing the same operation for all the parsed HTML pieces of codes
    for n in all_conditions: 
        condition = n.find_next('span').find_next('span').find_next('span').text.replace('\n', '').replace('  ','')
        condition = condition
        list_conditions.append(condition)
        
import pandas as pd
df = pd.DataFrame({'link': list_links,
'price': list_prices,
'shipping': list_shipping_prices,
'condition': list_conditions,
'country': list_countries,
})
print(df.info())
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
link         10 non-null object
price        10 non-null object
shipping     10 non-null object
condition    10 non-null object
country      10 non-null object
dtypes: object(5)
memory usage: 528.0+ bytes
None
                                          link  price  shipping  \
0    http://www.discogs.com/sell/item/13131222  $2.89    $29.99   
1  http://www.discogs.com/sell/item/1200745159  €4.00     €8.50   
2   http://www.discogs.com/sell/item/756711589  €5.00    €13.00   
3   http://www.discogs.com/sell/item/976347912  €5.00    €13.00   
4  http://www.discogs.com/sell/item/1198018404  €5.00    €13.00   
5     http://www.discogs.com/sell/item/1792259  $6.00  shipping   
6   http://www.discogs.com/sell/item/928280330  £5.00     £8.00   
7  http://www.discogs.com/sell/item/1194013845  €7.69     €3.90   
8  http://www.discogs.com/sell/item/1193689380  €9.88     €3.90   
9   http://www.disc

In [2]:
from datetime import datetime
import os

date_today_str = datetime.today().strftime('%Y-%m-%d')

path = '/Users/julien.varin/Downloads/discogs_watcher/export-'

file_name = date_today_str+'.csv'

In [3]:
from datetime import date, timedelta

date_today = date.today().strftime('%Y-%m-%d')
date_yesterday = (date.today() - timedelta(days=1)).strftime('%Y-%m-%d')

with open(path+date_today+'.csv', 'r') as t1, open(path+date_yesterday+'.csv', 'r') as t2:
    file_today = t1.readlines()
    file_yesterday = t2.readlines()

# returns all the rows today's file that were not in yesterday's file
with open('update.csv', 'w') as outFile:
    for line in file_today:
        if line not in file_yesterday:
            outFile.write(line)
            
colnames = ['link', 'price', 'shipping', 'conditiom', 'ships from'] 
pd.read_csv('update.csv', names=colnames, header=None)

,link,price,shipping,conditiom,ships from
0,http://www.discogs.com/sell/item/976347912,€5.00,€13.00,Near Mint (NM or M-),France
1,http://www.discogs.com/sell/item/1198018404,€5.00,€13.00,Near Mint (NM or M-),Netherlands
2,http://www.discogs.com/sell/item/1792259,$6.00,shipping,Near Mint (NM or M-),Switzerland
3,http://www.discogs.com/sell/item/349428214,€4.00,€11.00,Very Good Plus (VG+),Spain


In [19]:
import csv
from tabulate import tabulate
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

import smtplib

me = 'julien.varin.fr@gmail.com'
password = 'xx'
server = 'smtp.gmail.com:587'
you = 'julien.varin.fr@gmail.com'

text = """
New stuff:
{table}
"""

html = """
<html><body>
<p>New stuff:</p>
{table}
</body></html>
"""

with open('update.csv') as input_file:
    reader = csv.reader(input_file)
    data = list(reader)
    #print(data)
text = text.format(table=tabulate(data, headers=colnames, tablefmt="grid"))
print(text)
html = html.format(table=tabulate(data, headers=colnames, tablefmt="html"))
# print(html)

message = MIMEMultipart( "alternative", None, [MIMEText(text), MIMEText(html,'html')])
message['Subject'] = "test"
message['From'] = me
message['To'] = you
server = smtplib.SMTP(server)
server.ehlo()
server.starttls()
server.login(me, password)
server.sendmail(me, you, message.as_string())
server.quit()

print('Email sent!')



New stuff:
+---------------------------------------------+---------+------------+----------------------+--------------+
| link                                        | price   | shipping   | conditiom            | ships from   |
+=============================================+=========+============+======================+==============+
| http://www.discogs.com/sell/item/976347912  | €5.00   | €13.00     | Near Mint (NM or M-) | France       |
+---------------------------------------------+---------+------------+----------------------+--------------+
| http://www.discogs.com/sell/item/1198018404 | €5.00   | €13.00     | Near Mint (NM or M-) | Netherlands  |
+---------------------------------------------+---------+------------+----------------------+--------------+
| http://www.discogs.com/sell/item/1792259    | $6.00   | shipping   | Near Mint (NM or M-) | Switzerland  |
+---------------------------------------------+---------+------------+----------------------+--------------+
| http: